In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import splitfolders

In [ ]:
splitfolders.ratio("dataset", output="dataset_output", seed=42)

In [ ]:
dataset_path = "dataset_output"
test_set = keras.utils.image_dataset_from_directory(os.path.join(dataset_path, 'test'), image_size=(224, 224), shuffle=True, seed=42)
train_set = keras.utils.image_dataset_from_directory(os.path.join(dataset_path, 'train'), image_size=(224, 224), shuffle=True, seed=42)
val_set = keras.utils.image_dataset_from_directory(os.path.join(dataset_path, 'val'), image_size=(224, 224), shuffle=True, seed=42)

In [ ]:
def preprocess_img(img, label):
  img = keras.applications.mobilenet_v2.preprocess_input(img)
  return img, label

In [ ]:
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess_img).prefetch(1)
val_set = val_set.map(preprocess_img).prefetch(1)
test_set = test_set.map(preprocess_img).prefetch(1)

In [ ]:
input_ = keras.layers.Input((224, 224, 3))
data_augmentation = keras.models.Sequential([
  keras.layers.RandomFlip('horizontal'),
  keras.layers.RandomRotation(0.3),
  keras.layers.RandomZoom(0.3, 0.3)
])

In [ ]:
base_model = keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), include_top=False)

In [ ]:
head_model = data_augmentation(input_)
head_model = base_model(head_model)
head_model = keras.layers.GlobalAveragePooling2D()(head_model)
head_model = keras.layers.Dense(1, activation='sigmoid')(head_model)
model = keras.models.Model(inputs=input_, outputs=head_model)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
optimizer = keras.optimizers.Nadam()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_set, epochs=10, validation_data=val_set)

In [ ]:
fig, ax = plt.subplots()
ax.plot(history.history['loss'], label='train loss')
ax.plot(history.history['val_loss'], label='valid loss')
ax.set_xlabel('# of epochs')
ax.set_ylabel('Loss')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.plot(history.history['accuracy'], label='train accuracy')
ax.plot(history.history['val_accuracy'], label='valid accuracy')
ax.set_xlabel('# of epochs')
ax.set_ylabel('Accuracy')
ax.legend()

In [ ]:
for layer in base_model.layers:
  layer.trainable = True

In [ ]:
low_lr_optimizer = keras.optimizers.SGD(0.001)
# low_lr_optimizer = keras.optimizers.Nadam(0.0001)
model.compile(optimizer=low_lr_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
all_history = model.fit(train_set, epochs=10, validation_data=val_set)

In [ ]:
fig, ax = plt.subplots()
ax.plot(all_history.history['loss'], label='train loss')
ax.plot(all_history.history['val_loss'], label='valid loss')
ax.set_xlabel('# of epochs')
ax.set_ylabel('Loss')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.plot(all_history.history['accuracy'], label='train accuracy')
ax.plot(all_history.history['val_accuracy'], label='valid accuracy')
ax.set_xlabel('# of epochs')
ax.set_ylabel('Accuracy')
ax.legend()

In [ ]:
model.save('face_mask_detection_model')